In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import img_to_array,load_img
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report,confusion_matrix
from tensorflow.keras.preprocessing import image 
from sklearn.model_selection import train_test_split

In [15]:
train_genrator = ImageDataGenerator(rescale=1/255, 
                                   horizontal_flip=True,
                                   zoom_range=0.2,
                                    shear_range=0.2)
train_data = train_genrator.flow_from_directory("../input/real-life-industrial-dataset-of-casting-product/casting_data/casting_data/train",
                                               class_mode="binary",
                                               batch_size=32,
                                               target_size =(64,64),
                                               color_mode = "grayscale")

test_genrator = ImageDataGenerator(rescale=1/255)
test_data = train_genrator.flow_from_directory("../input/real-life-industrial-dataset-of-casting-product/casting_data/casting_data/test",
                                               class_mode="binary",
                                               batch_size=32,
                                               target_size =(64,64),
                                               color_mode = "grayscale")

In [9]:
#Building the model 
#2 conv layers with kernel size of 3x3 followed bya maxpool with stride of 2 then 2 conv layers again 
#followed by maxpool of stride of 2 with pool size of 2x2 followd by 2 dense layers with activation 
# of relu and then a dense layer with 1 neuron with activation as sigmoid

Layers = [keras.layers.Conv2D(filters=16,kernel_size=(3,3),padding="same",input_shape=(64,64,1)),
         #Input is grayscale thus only one channel in input shape
          keras.layers.Activation("relu"),
          keras.layers.Conv2D(filters=16,kernel_size=(3,3),padding="same"),
          keras.layers.Activation("relu"),
          keras.layers.MaxPooling2D(pool_size=(2,2),strides=2),
          keras.layers.Conv2D(filters=16,kernel_size=(3,3),padding="same"),
          keras.layers.Activation("relu"),
          keras.layers.Conv2D(filters=16,kernel_size=(3,3),padding="same"),
          keras.layers.Activation("relu"),
          keras.layers.MaxPooling2D(pool_size=(2,2),strides=2),
          keras.layers.Flatten(),
          keras.layers.Dense(128),
          keras.layers.Activation("relu"),
          keras.layers.Dense(128),
          keras.layers.Activation("relu"),
          keras.layers.Dense(1,activation="sigmoid")]


            

In [10]:
# Adding layers to our sequential model
model = keras.models.Sequential(layers=Layers)

In [11]:
model.summary()

In [13]:
model.compile(optimizer="sgd",loss="binary_crossentropy",metrics=["accuracy"])

In [20]:
history = model.fit_generator(train_data,validation_data=test_data,epochs=20,verbose=2)

In [21]:
model.save("my_submission_model.h5")

In [25]:
def predictions(image_address):
    test_image = image.load_img(str(image_address),target_size=(64,64),color_mode= "grayscale")
    test_image = image.img_to_array(test_image)
    test_image = test_image/255
    test_image = np.expand_dims(test_image,axis=0)
    result = model.predict(test_image)
    if result[0]<=0.5:
        print("Defective")
    else:
        print("Not Defective ; ie:OK")

In [26]:

## Address from not defective 
address = "../input/real-life-industrial-dataset-of-casting-product/casting_data/casting_data/test/ok_front/cast_ok_0_1026.jpeg"
predictions(image_address=address)

Predictions is correct

In [27]:
## Address from defective 

address = "../input/real-life-industrial-dataset-of-casting-product/casting_data/casting_data/test/def_front/cast_def_0_1189.jpeg"
predictions(address)

Prediction is correct